In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
import pickle

import tensorflow as tf
from tensorflow import keras
from keras.models import Model,Sequential,load_model
from keras.layers import Input, Embedding
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import LSTM
import keras.metrics as metrics
import itertools
from tensorflow.python.keras.utils.data_utils import Sequence
from decimal import Decimal
from keras import backend as K
from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense

In [2]:
inp=pd.read_csv("../PJ sensor.csv",usecols=[6,7,10,11])
out=pd.read_csv("../PJ sensor.csv",usecols=[2,3,4,5,8,9])

In [3]:
inp.head(5)

,Temperature,Relative Humidity,date,Time
0,31,65,191213,175844
1,31,65,191213,175904
2,31,65,191213,175924
3,31,65,191213,175944
4,31,66,191213,180004


In [4]:
out.head(5)

,NO2,O3,CO,SO2,PM2.5,PM10
0,-15.075377,35.714286,1766.233766,148.484848,11.229150,17.323938
1,-20.100503,41.208791,1733.766234,142.424242,9.413914,24.967407
2,-15.075377,43.956044,1711.038961,136.363636,10.350110,56.781536
3,0.000000,57.692308,1688.311688,139.393939,10.488765,30.371115
4,-20.100503,54.945055,1681.818182,130.303030,12.353048,188.026886


In [5]:
inp=np.array(inp)
out=np.array(out)

In [6]:
from sklearn.preprocessing import MinMaxScaler
import warnings
scaler_obj=MinMaxScaler()
X1=scaler_obj.fit_transform(inp)
Y1=scaler_obj.fit_transform(out)

warnings.filterwarnings(action='ignore', category=UserWarning)

In [11]:
# Splitting Data into training and testing dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)

from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

#creating object of sgboostregressor
model1=MultiOutputRegressor(XGBRegressor(tree_method='gpu_hist', gpu_id=0,objective ='reg:squarederror',max_depth=9,cosample_bytree=.5,learning_rate=.1,min_child_weight=3,
                   n_estimators=450,subsample=.7))
#training the model
# model_fit1=model1.fit(x_train,y_train)


eval_set = [(x_train, y_train), (x_test, y_test)]
eval_metric = ["auc","error"]
model_fit1=model1.fit(x_train, y_train)
%time model_fit1
print("Model training is Done!!")

filename1 = 'xgboost.sav'
pickle.dump(model_fit1, open(filename1, 'wb'))

[09:39:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:40:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[09:40:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be use

In [12]:
from sklearn.metrics import r2_score
from sklearn import metrics
train_sizes=['NO2','O3','NO','CO','PM1','PM2.5','PM10']

#finding out the r2 score
y_train_pred1=model1.predict(x_train)
r2_train1=r2_score(y_train,y_train_pred1,multioutput='variance_weighted')

y_test_pred1=model1.predict(x_test)
r2_test1=r2_score(y_test,y_test_pred1,multioutput='variance_weighted')

print('r2 score on train data '+str(r2_train1))
print('r2 score on test data '+ str(r2_test1))

xgboost_mae=metrics.mean_absolute_error(y_test, y_test_pred1)
xgboost_mse=metrics.mean_squared_error(y_test, y_test_pred1)
xgboost_rmse=np.sqrt(xgboost_mse)
print('Mean Absolute Error:',xgboost_mae)
print('Mean Squared Error:',xgboost_mse )
print('Root Mean Squared Error:',xgboost_rmse)
print(' \n')


r2 score on train data 0.804894044676069
r2 score on test data 0.7867491776691122
Mean Absolute Error: 0.006571193824432935
Mean Squared Error: 0.0001733531624915415
Root Mean Squared Error: 0.013166364816893898
 



In [7]:
import pickle
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)
loaded_model_fit7 = pickle.load(open("xgboost.sav", 'rb'))
y_test_pred=loaded_model_fit7.predict(x_test)
print("Predicted :\n",y_test_pred)
print("\n")
r2_test=r2_score(y_test,y_test_pred,multioutput='variance_weighted')
print("R2 Score : ",r2_test)

Predicted :
 [[0.48340052 0.5991475  0.04695781 0.0085996  0.06914376 0.00457146]
 [0.4310623  0.57405657 0.02534223 0.00805853 0.04595666 0.00399989]
 [0.48590872 0.6038791  0.05655789 0.00966027 0.10579658 0.00713156]
 ...
 [0.40851507 0.550818   0.03021315 0.01014406 0.06170817 0.00457344]
 [0.40562367 0.56409824 0.03133747 0.01069754 0.07458849 0.00595892]
 [0.47769344 0.56437266 0.06263287 0.00888559 0.09093509 0.00729026]]


R2 Score :  0.7896089086526633


In [ ]:
#completed